In [ ]:
import sys
sys.path.append('../src')

# if run in a notebook, generate a matching python script",
if 'ipykernel' in sys.modules:
    %reload_ext autoreload
    %autoreload 2


In [ ]:
import cv2 as cv
from cellpose import models, io, plot

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pickle
import re
import tifffile as tiff
from time import time
from tqdm import tqdm
from utils import list_subdir_filter as lsd, unique, get_id
from metadata import metadata

global md
md = metadata()

model = models.Cellpose(gpu=True, model_type='nuclei')

In [ ]:
def segment_image(img_file):

    tissue_id = re.sub('.*A40.([0-9]{4}).*$', '\\1', img_file)
    print('\n\n', tissue_id)
    if len(lsd(seg_dir, False, f'A40_{tissue_id}_dapi1_[0-9]\.npy')) == 10:
        print(f'{tissue_id} already segmented. Skipping...')
        return None
    
    #img = np.load(img_file)[0]
    img = tiff.imread(img_file, key=0)
    img = (img >> 8).astype('uint8')
    max_size = np.shape(img)[0]
    width = 4000
    
    for i in tqdm(range((max_size // width) + 1)):
        print(i)
        min_row, max_row = max(0, i * width - 100), min(max_size, (i+1) * width + 100)
        masks, flows, styles, diams = model.eval(img[min_row:max_row], diameter=None, channels=[0,0])
        # pickle_dict = {'masks':masks, 'flows':flows, 'styles':styles, 'diams':diams}
        
        np.save(os.path.join(seg_dir, f'A40_{tissue_id}_{i}_dapi1.npy'), masks)
        # with open(os.path.join(seg_dir, f'A40_{tissue_id}_dapi1_{i:02}.pkl'), 'wb') as handle:
        #     pickle.dump(pickle_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
seg_dir = os.path.join(md.folders['segmented'], 'cellpose')
Path(seg_dir).mkdir(exist_ok=True, parents=True)

all_samples = lsd(os.path.join(md.folders['images'], 'mouse'))
all_img_files = lsd(md.folders['images'], True, '.tif')
all_seg_files = lsd(seg_dir, False, 'A40_[0-9]{4}')

all_sample_ids, all_img_ids, all_seg_ids = list(map(
    lambda x: unique([get_id(f) for f in x]), 
    [all_samples, all_img_files, all_seg_files]
))

to_segment = [f for f in all_img_files]



In [ ]:
for img_file in to_segment:
    segment_image(img_file)